In [2]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from gensim.models import Word2Vec 
from itertools import islice
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

In [3]:
def get_new_data(data):
    cw = lambda x: [list(i.split(',')) for i in list(x.split('[sep]'))]
    data['sentences'] = data['text'].apply(cw)
    cw = lambda x: ','.join([j for i in x for j in i])
    data['words'] = data['sentences'].apply(cw)
    creative_list = []
    ad_id_list = []
    product_id_list = []
    product_category_list = []
    advertiser_id_list = []
    industry_list = []

    for _, row in data.iterrows():
        sentences = row['sentences']
        creative_str = ','.join(sentences[0])
        ad_id_str = ','.join(sentences[1])
        product_id_str = ','.join(sentences[2])
        product_category_str = ','.join(sentences[3])
        advertiser_id_str = ','.join(sentences[4])
        industry_str = ','.join(sentences[5])
        creative_list.append(creative_str)
        ad_id_list.append(ad_id_str)
        product_id_list.append(product_id_str)
        product_category_list.append(product_category_str)
        advertiser_id_list.append(advertiser_id_str)
        industry_list.append(industry_str)

    data = pd.DataFrame({'user_id': data['user_id'],
                         'text': data['text'],
                         'sentences': data['sentences'],
                         'words': data['words'],
                         'creative_id': creative_list,
                         'ad_id': ad_id_list,
                         'product_id': product_id_list,
                         'product_category':product_category_list,
                         'advertiser_id': advertiser_id_list,
                         'industry': industry_list,
                         'types':data['types']
                         })

    return data


#生成muti-count-encode
train = pd.read_csv('../data/train/train_data.csv')
test = pd.read_csv('../data/test/test_data.csv')
train['types'] = 'train'
test['types'] = 'test'
data = pd.concat([train,test])
data = get_new_data(data)
data = data.reset_index()

In [ ]:
#生成muti-count-encode
cv = CountVectorizer()
wordlist = set([word for l in data['words'].values for word in l])
wordlist.remove('')
cv.fit(wordlist)
with open('../model/other/muti-count-encode.pickle', 'wb') as f:
    pickle.dump(cv, f)

In [ ]:
#生成word2vec-encode


sentence = [i for t in data['sentences'].values for i in t]
model = Word2Vec(sentence, size=300, window=5, min_count=1, workers=60, sg=1)
model.save('../data/other/word2vec.model')
#model.wv.save_word2vec_format('../model/other/word2vec.bin',binary = True)

#model = gensim.models.Word2Vec.load('../model/other/word2vec.txt')


In [ ]:
def topk(t,thread,doc):
    return ','.join(list(islice([doc[w] for w,score in t],thread)))


def generate_keywords_text(data,key):
    t = data[key].values
    t = [i.split(',') for i in t]
    doc = Dictionary(t)
    corpus = [doc.doc2bow(i) for i in t]
    model = TfidfModel(corpus)
    return [topk(i,50,doc) for i in model[corpus]]

In [ ]:
data['creative_id_key'] = generate_keywords_text(data,'creative_id')

In [ ]:
data['ad_id_key'] = generate_keywords_text(data,'ad_id')
data['product_id_key'] = generate_keywords_text(data,'product_id')
data['product_category_key'] = generate_keywords_text(data,'product_category')
data['advertiser_id_key'] = generate_keywords_text(data,'advertiser_id')
data['industry_key'] = generate_keywords_text(data,'industry')
data.to_csv('../data/data.csv',index=False)

In [ ]:
data.head(5)